In [ ]:
!pip install -q torch transformers rouge-score sentence-transformers datasets huggingface-hub

from transformers import pipeline
from rouge_score import rouge_scorer
import pandas as pd



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.6 MB/s eta 0:00:00


In [ ]:
# Load summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
articles = []
print("Please paste 5 long news articles one by one. After pasting each article, type 'END' on a new line.")

for i in range(1, 6):
    print(f"\nPaste article #{i}:")
    lines = []
    while True:
        line = input()
        if line.strip() == "END":
            break
        lines.append(line)
    article_text = "\n".join(lines)
    articles.append(article_text)

Please paste 5 long news articles one by one. After pasting each article, type 'END' on a new line.

Paste article #1:
Nvidia will launch a new artificial intelligence chipset for China at a significantly lower price than its recently restricted H20 model and plans to start mass production as early as June, sources familiar with the matter said.  The GPU or graphics processing unit will be part of Nvidia’s latest generation Blackwell-architecture AI processors and is expected to be priced between $6,500 and $8,000, well below the $10,000-$12,000 the H20 sold for, according to two of the sources.  The lower price reflects its weaker specifications and simpler manufacturing requirements.  It will be based on Nvidia’s RTX Pro 6000D, a server-class graphics processor and will use conventional GDDR7 memory instead of more advanced high bandwidth memory, the two sources said. The new chip’s price, specifications and production timing have not previously been reported.  The three sources Reut

In [ ]:
# Define prompt templates for zero-shot summarization variations
prompt_templates = [
    {"prompt": "Summarize the given news article in 3–4 complete and well-structured sentences suitable for a college-level analysis. Ensure the summary includes the main idea, key developments or events, and any significant outcomes or implications. Use formal language and avoid vague phrasing. The summary should reflect a clear understanding of the topic and provide enough context for readers unfamiliar with the article.", "max_length": 100, "min_length": 60},
    {"prompt": "Briefly summarize the following news article in 1–2 concise sentences. Ensure the summary captures the main point or event and includes the most important facts or developments. Additionally, provide 2–3 key bullet points highlighting critical details such as names, dates, statistics, decisions, or consequences mentioned in the article. The summary should be easy to understand for someone unfamiliar with the topic, and the bullet points should emphasize the most relevant information without repeating the summary.", "max_length": 80, "min_length": 40},
    {"prompt": "Write a comprehensive and detailed summary of the article, consisting of 5 to 7 well-structured sentences. Focus on highlighting the key facts, main events, and important figures or organizations involved. Be sure to explain the context of the news clearly, including any background information necessary to understand the topic. Provide a clear and informative explanation of why the news is important, what its potential impact might be, and how it relates to current trends or issues. Avoid vague statements and aim to convey the core message of the article in a concise but thorough manner. Make the summary easy to understand for someone who has no prior knowledge of the topic.", "max_length": 150, "min_length": 100},
]

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

results = []

for idx, text in enumerate(articles, 1):
    print(f"\nProcessing Article #{idx}...")
    for pt in prompt_templates:
        input_text = pt['prompt'] + "\n\n" + text

        # Truncate input to fit within model's token limit
        inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=1024)
        decoded_input = tokenizer.decode(inputs[0], skip_special_tokens=True)

        summary = summarizer(decoded_input, max_length=pt['max_length'], min_length=pt['min_length'], do_sample=False)[0]['summary_text']
        scores = scorer.score(text, summary)

        results.append({
            "article_num": idx,
            "prompt": pt['prompt'],
            "max_length": pt['max_length'],
            "min_length": pt['min_length'],
            "summary": summary,
            "rouge1_f1": scores['rouge1'].fmeasure,
            "rouge2_f1": scores['rouge2'].fmeasure,
            "rougeL_f1": scores['rougeL'].fmeasure
        })

        print(f"Prompt: {pt['prompt']}")
        print(f"Summary:\n{summary}\n")
        print(f"ROUGE-1 F1: {scores['rouge1'].fmeasure:.4f}, ROUGE-2 F1: {scores['rouge2'].fmeasure:.4f}, ROUGE-L F1: {scores['rougeL'].fmeasure:.4f}")
        print("-" * 60)



Processing Article #1...
Prompt: Summarize the given news article in 3–4 complete and well-structured sentences suitable for a college-level analysis. Ensure the summary includes the main idea, key developments or events, and any significant outcomes or implications. Use formal language and avoid vague phrasing. The summary should reflect a clear understanding of the topic and provide enough context for readers unfamiliar with the article.
Summary:
N Nvidia will launch a new artificial intelligence chipset for China at a significantly lower price than its recently restricted H20 model. The new chip is expected to be priced between $6,500 and $8,000, well below the $10,000-$12,000 the H20 sold for. It will be based on Nvidia’s RTX Pro 6000D, a server-class graphics processor.

ROUGE-1 F1: 0.1854, ROUGE-2 F1: 0.1738, ROUGE-L F1: 0.1793
------------------------------------------------------------
Prompt: Briefly summarize the following news article in 1–2 concise sentences. Ensure the su

In [ ]:
df = pd.DataFrame(results)
df.to_csv("zero_shot_summarization_results.csv", index=False)
print("\nSaved all summaries and ROUGE scores to 'zero_shot_summarization_results.csv'")



Saved all summaries and ROUGE scores to 'zero_shot_summarization_results.csv'
